In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

In [2]:
dataName = 'WCup2022_TeamData.csv'
DFTeamData = pd.read_csv(dataName)
DFTeamData

,Team,Group,Area,Ranking
0,Qatar,A,Host,50
1,Ecuador,A,SouthAmerica,44
2,Senegal,A,Africa,18
3,Netherlands,A,Europe,8
4,England,B,Europe,5
5,Iran,B,Asia,20
6,USA,B,Caribbean,16
7,Wales,B,Europe,19
8,Argentina,C,SouthAmerica,3
9,SaudiArabia,C,Asia,51


In [3]:
def Ranking(RoundRobin):
    Team_list = RoundRobin.index.tolist()
    
    for team_index in Team_list:
        scored_sum = 0
        conceded_sum = 0
        winningPoint = 0
        for team_columns in Team_list:
            score = RoundRobin.at[team_index, team_columns]
            if type(score) is float:
                continue
            score_list = score.split(',')
            if len(score_list) != 2:
                continue
            
            scored, conceded = int(score_list[0]), int(score_list[1])
            scored_sum += scored
            conceded_sum += conceded
            if scored > conceded: winningPoint += 3
            elif scored == conceded: winningPoint += 1
                
        RoundRobin.at[team_index, 'WinningPoints'] = winningPoint
        RoundRobin.at[team_index, 'Score'] = scored_sum
        RoundRobin.at[team_index, 'Conceded'] = conceded_sum
        RoundRobin.at[team_index, 'GoalDiff'] = scored_sum - conceded_sum
        
    WinningPoints_rank = RoundRobin['WinningPoints'].rank(ascending=False, method='min')
    WinningPoints_list = WinningPoints_rank.tolist()
    RoundRobin['Ranking'] = WinningPoints_list
    Duplication_list1 = []
    for i in range(1,5):
        if WinningPoints_list.count(i) >= 2:
            Duplication_list1.append(i)
    for duplication1 in Duplication_list1:
        RoundRobin_duplication1 = RoundRobin.loc[RoundRobin['Ranking'] == duplication1]
        GoalDiff_rank = RoundRobin_duplication1['GoalDiff'].rank(ascending=False, method='min')
        GoalDiff_list = GoalDiff_rank.tolist()
        GoalDiff_index = GoalDiff_rank.index.tolist()
        for j in range(1,len(GoalDiff_list)+1):
            if GoalDiff_list.count(j) == 1:
                RoundRobin.at[GoalDiff_index[GoalDiff_list.index(j)], 'Ranking'] = j + duplication1-1
            elif GoalDiff_list.count(j) >= 2:
                idx = -1
                Duplication_list2 = []
                for cnt in range(GoalDiff_list.count(j)):
                    idx = GoalDiff_list.index(j, idx+1)
                    Duplication_list2.append(GoalDiff_index[idx])
                RoundRobin.loc[Duplication_list2, 'Ranking'] = j + duplication1-1
                RoundRobin_duplication2 = RoundRobin.loc[Duplication_list2]
                Score_rank = RoundRobin_duplication2['Score'].rank(ascending=False, method='min')
                Score_list = Score_rank.tolist()
                Score_index = Score_rank.index.tolist()
                for k in range(1,len(Score_list)+1):
                    if Score_list.count(k) == 1:
                        RoundRobin.at[Score_index[Score_list.index(k)], 'Ranking'] = k + j-1 + duplication1-1
    
    
    return RoundRobin

In [4]:
def RankingProbability(RoundRobin):
    Team_list = RoundRobin.index.tolist()
    #Score_list = ['5,0', '4,0', '3,0', '2,0', '1,0', '0,0', '0,1', '0,2', '0,3', '0,4', '0,5']
    #Score_list = ['1,0', '0,0', '0,1']
    # 11パターンで240秒(7分), 1秒で約670回
    # 予測：5試合分　13パターンで555秒x13=2時間
    # 予測：4試合分　19パターンで9分,25パターンで28分
    Score_list        = ['0,0', '1,1', '2,2', '3,0', '3,1', '2,1', '2,0', '1,0', '0,1', '0,2', '1,2', '0,3', '1,3']
    Score_inverselist = ['0,0', '1,1', '2,2', '0,3', '1,3', '1,2', '0,2', '0,1', '1,0', '2,0', '2,1', '3,0', '3,1']
    X = np.array(range(len(Score_list)))
    
    Ranking_num = pd.DataFrame(0, columns=range(1,5), index=Team_list)
    Ranking_probability = pd.DataFrame(columns=range(1,5), index=Team_list)
    print("start : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
    continueSwitch1 = 0
    for i1 in X:
        if continueSwitch1 == 1: continue
        print("0,1(" + str(i1) + ") : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
        RoundRobin1 = RoundRobin.copy()
        if RoundRobin1.iat[0,1] is float(np.nan):
            RoundRobin1.iat[0,1] = Score_list[i1]
            RoundRobin1.iat[1,0] = Score_inverselist[i1]
        else: continueSwitch1 = 1
        continueSwitch2 = 0
        for i2 in X:
            if continueSwitch2 == 1: continue
            #print("0,2 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
            RoundRobin2 = RoundRobin1.copy()
            if RoundRobin2.iat[0,2] is float(np.nan):
                RoundRobin2.iat[0,2] = Score_list[i2]
                RoundRobin2.iat[2,0] = Score_inverselist[i2]
            else: continueSwitch2 = 1
            continueSwitch3 = 0
            for i3 in X:
                if continueSwitch3 == 1: continue
                #print("0,3 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                RoundRobin3 = RoundRobin2.copy()
                if RoundRobin3.iat[0,3] is float(np.nan):
                    RoundRobin3.iat[0,3] = Score_list[i3]
                    RoundRobin3.iat[3,0] = Score_inverselist[i3]
                else: continueSwitch3 = 1
                continueSwitch4 = 0
                for i4 in X:
                    if continueSwitch4 == 1: continue
                    #print("1,2 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                    RoundRobin4 = RoundRobin3.copy()
                    if RoundRobin4.iat[1,2] is float(np.nan):
                        RoundRobin4.iat[1,2] = Score_list[i4]
                        RoundRobin4.iat[2,1] = Score_inverselist[i4]
                    else: continueSwitch4 = 1
                    continueSwitch5 = 0
                    for i5 in X:
                        if continueSwitch5 == 1: continue
                        #print("1,3 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                        RoundRobin5 = RoundRobin4.copy()
                        if RoundRobin5.iat[1,3] is float(np.nan):
                            RoundRobin5.iat[1,3] = Score_list[i5]
                            RoundRobin5.iat[3,1] = Score_inverselist[i5]
                        else: continueSwitch5 = 1
                        continueSwitch6 = 0
                        for i6 in X:
                            if continueSwitch6 == 1: continue
                            #print("2,3 : " + dt.now().strftime("%Y.%m.%d %H:%M:%S"))
                            RoundRobin6 = RoundRobin5.copy()
                            if RoundRobin6.iat[2,3] is float(np.nan):
                                RoundRobin6.iat[2,3] = Score_list[i6]
                                RoundRobin6.iat[3,2] = Score_inverselist[i6]
                            else: continueSwitch6 = 1
                                
                            RoundRobin_rank = Ranking(RoundRobin6.copy())
                            
                            for j in range(1,5):
                                for k in range(0,4):
                                    if RoundRobin_rank.at[Team_list[k], 'Ranking'] == j:
                                        Ranking_num.at[Team_list[k], j] += 1
                            
    
    for k in range(0,4):
        patternNum = Ranking_num.loc[Team_list[k], :].sum()
        for j in range(1,5):
            Ranking_probability.at[Team_list[k], j] = str((Ranking_num.at[Team_list[k], j]/patternNum*100).round(1)) + '%'
                            
    return Ranking_num, Ranking_probability

In [5]:
Group_E_Team = DFTeamData.loc[DFTeamData['Group'] == 'E']
Group_E_Team_list = Group_E_Team['Team'].tolist()
Group_E_RoundRobin = pd.DataFrame(columns=Group_E_Team_list, index=Group_E_Team_list)
for team in Group_E_Team_list:
    Group_E_RoundRobin.at[team, team] = "-"

In [6]:
Ranking_JapanPropability = pd.DataFrame(columns=range(1,5))

In [7]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '0,0' , '0,0'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['0,0', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

start : 2022.11.23 19:04:48
0,1(0) : 2022.11.23 19:04:48
0,1(1) : 2022.11.23 19:05:40
0,1(2) : 2022.11.23 19:06:33
0,1(3) : 2022.11.23 19:07:23
0,1(4) : 2022.11.23 19:08:00
0,1(5) : 2022.11.23 19:08:39
0,1(6) : 2022.11.23 19:09:17
0,1(7) : 2022.11.23 19:09:56
0,1(8) : 2022.11.23 19:10:34
0,1(9) : 2022.11.23 19:11:13
0,1(10) : 2022.11.23 19:11:51
0,1(11) : 2022.11.23 19:12:30
0,1(12) : 2022.11.23 19:13:07


,1,2,3,4
Spain,29.7%,22.6%,22.5%,25.2%
CostaRica,29.7%,22.6%,22.5%,25.2%
Germany,21.3%,27.7%,27.1%,24.0%
Japan,21.3%,27.7%,27.1%,24.0%


In [8]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,1' , '1,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['1,1', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

start : 2022.11.23 19:13:46
0,1(0) : 2022.11.23 19:13:46
0,1(1) : 2022.11.23 19:14:36
0,1(2) : 2022.11.23 19:15:26
0,1(3) : 2022.11.23 19:16:15
0,1(4) : 2022.11.23 19:16:53
0,1(5) : 2022.11.23 19:17:31
0,1(6) : 2022.11.23 19:18:10
0,1(7) : 2022.11.23 19:18:48
0,1(8) : 2022.11.23 19:19:25
0,1(9) : 2022.11.23 19:20:03
0,1(10) : 2022.11.23 19:20:40
0,1(11) : 2022.11.23 19:21:18
0,1(12) : 2022.11.23 19:21:55


,1,2,3,4
Spain,29.4%,22.3%,22.8%,25.5%
CostaRica,29.4%,22.3%,22.8%,25.5%
Germany,21.7%,28.0%,26.6%,23.7%
Japan,21.7%,28.0%,26.6%,23.7%


In [9]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '2,2' , '2,2'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['2,2', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

start : 2022.11.23 19:22:32
0,1(0) : 2022.11.23 19:22:32
0,1(1) : 2022.11.23 19:23:21
0,1(2) : 2022.11.23 19:24:10
0,1(3) : 2022.11.23 19:24:59
0,1(4) : 2022.11.23 19:25:36
0,1(5) : 2022.11.23 19:26:14
0,1(6) : 2022.11.23 19:26:51
0,1(7) : 2022.11.23 19:27:29
0,1(8) : 2022.11.23 19:28:06
0,1(9) : 2022.11.23 19:28:44
0,1(10) : 2022.11.23 19:29:22
0,1(11) : 2022.11.23 19:29:59
0,1(12) : 2022.11.23 19:30:36


,1,2,3,4
Spain,29.0%,21.9%,23.2%,25.9%
CostaRica,29.0%,21.9%,23.2%,25.9%
Germany,22.0%,28.3%,26.3%,23.4%
Japan,22.0%,28.3%,26.3%,23.4%


In [10]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '0,3' , '3,0'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['3,0', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

start : 2022.11.23 19:31:14
0,1(0) : 2022.11.23 19:31:14
0,1(1) : 2022.11.23 19:31:51
0,1(2) : 2022.11.23 19:32:28
0,1(3) : 2022.11.23 19:33:05
0,1(4) : 2022.11.23 19:33:48
0,1(5) : 2022.11.23 19:34:30
0,1(6) : 2022.11.23 19:35:11
0,1(7) : 2022.11.23 19:35:53
0,1(8) : 2022.11.23 19:36:35
0,1(9) : 2022.11.23 19:37:16
0,1(10) : 2022.11.23 19:37:58
0,1(11) : 2022.11.23 19:38:40
0,1(12) : 2022.11.23 19:39:23


,1,2,3,4
Spain,22.6%,27.5%,27.2%,22.7%
CostaRica,22.6%,27.5%,27.2%,22.7%
Germany,4.2%,16.3%,29.7%,49.9%
Japan,52.5%,29.1%,15.6%,2.9%


In [11]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,2' , '2,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['2,1', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

start : 2022.11.23 19:40:05
0,1(0) : 2022.11.23 19:40:05
0,1(1) : 2022.11.23 19:40:43
0,1(2) : 2022.11.23 19:41:21
0,1(3) : 2022.11.23 19:42:00
0,1(4) : 2022.11.23 19:42:41
0,1(5) : 2022.11.23 19:43:24
0,1(6) : 2022.11.23 19:44:06
0,1(7) : 2022.11.23 19:44:48
0,1(8) : 2022.11.23 19:45:31
0,1(9) : 2022.11.23 19:46:14
0,1(10) : 2022.11.23 19:46:57
0,1(11) : 2022.11.23 19:47:39
0,1(12) : 2022.11.23 19:48:20


,1,2,3,4
Spain,25.3%,24.9%,24.4%,25.4%
CostaRica,25.3%,24.9%,24.4%,25.4%
Germany,8.8%,20.3%,30.9%,40.0%
Japan,43.1%,30.2%,19.7%,7.0%


In [12]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['2,0', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

start : 2022.11.23 19:49:03
0,1(0) : 2022.11.23 19:49:03
0,1(1) : 2022.11.23 19:49:40
0,1(2) : 2022.11.23 19:50:18
0,1(3) : 2022.11.23 19:50:55
0,1(4) : 2022.11.23 19:51:37
0,1(5) : 2022.11.23 19:52:21
0,1(6) : 2022.11.23 19:53:03
0,1(7) : 2022.11.23 19:53:47
0,1(8) : 2022.11.23 19:54:29
0,1(9) : 2022.11.23 19:55:11
0,1(10) : 2022.11.23 19:55:55
0,1(11) : 2022.11.23 19:56:37
0,1(12) : 2022.11.23 19:57:18


,1,2,3,4
Spain,24.5%,26.2%,25.7%,23.7%
CostaRica,24.5%,26.2%,25.7%,23.7%
Germany,6.1%,17.8%,30.3%,45.8%
Japan,47.4%,30.2%,17.7%,4.6%


In [13]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '0,1' , '1,0'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['1,0', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

start : 2022.11.23 19:58:02
0,1(0) : 2022.11.23 19:58:02
0,1(1) : 2022.11.23 19:58:39
0,1(2) : 2022.11.23 19:59:17
0,1(3) : 2022.11.23 19:59:55
0,1(4) : 2022.11.23 20:00:36
0,1(5) : 2022.11.23 20:01:18
0,1(6) : 2022.11.23 20:02:00
0,1(7) : 2022.11.23 20:02:42
0,1(8) : 2022.11.23 20:03:24
0,1(9) : 2022.11.23 20:04:09
0,1(10) : 2022.11.23 20:04:51
0,1(11) : 2022.11.23 20:05:34
0,1(12) : 2022.11.23 20:06:15


,1,2,3,4
Spain,25.9%,25.1%,24.2%,24.8%
CostaRica,25.9%,25.1%,24.2%,24.8%
Germany,8.4%,19.7%,30.7%,41.2%
Japan,41.9%,30.4%,20.3%,7.4%


In [14]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '1,0' , '0,1'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['0,1', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

start : 2022.11.23 20:06:58
0,1(0) : 2022.11.23 20:06:58
0,1(1) : 2022.11.23 20:07:36
0,1(2) : 2022.11.23 20:08:13
0,1(3) : 2022.11.23 20:08:51
0,1(4) : 2022.11.23 20:09:32
0,1(5) : 2022.11.23 20:10:13
0,1(6) : 2022.11.23 20:10:56
0,1(7) : 2022.11.23 20:11:38
0,1(8) : 2022.11.23 20:12:20
0,1(9) : 2022.11.23 20:13:03
0,1(10) : 2022.11.23 20:13:44
0,1(11) : 2022.11.23 20:14:27
0,1(12) : 2022.11.23 20:15:08


,1,2,3,4
Spain,25.9%,25.1%,24.2%,24.8%
CostaRica,25.9%,25.1%,24.2%,24.8%
Germany,41.9%,30.4%,20.3%,7.4%
Japan,8.4%,19.7%,30.7%,41.2%


In [15]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '2,0' , '0,2'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['0,2', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

start : 2022.11.23 20:15:49
0,1(0) : 2022.11.23 20:15:49
0,1(1) : 2022.11.23 20:16:26
0,1(2) : 2022.11.23 20:17:04
0,1(3) : 2022.11.23 20:17:40
0,1(4) : 2022.11.23 20:18:21
0,1(5) : 2022.11.23 20:19:04
0,1(6) : 2022.11.23 20:19:45
0,1(7) : 2022.11.23 20:20:28
0,1(8) : 2022.11.23 20:21:10
0,1(9) : 2022.11.23 20:21:51
0,1(10) : 2022.11.23 20:22:34
0,1(11) : 2022.11.23 20:23:15
0,1(12) : 2022.11.23 20:23:56


,1,2,3,4
Spain,24.5%,26.2%,25.7%,23.7%
CostaRica,24.5%,26.2%,25.7%,23.7%
Germany,47.4%,30.2%,17.7%,4.6%
Japan,6.1%,17.8%,30.3%,45.8%


In [16]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '2,1' , '1,2'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['1,2', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

start : 2022.11.23 20:24:39
0,1(0) : 2022.11.23 20:24:39
0,1(1) : 2022.11.23 20:25:15
0,1(2) : 2022.11.23 20:25:52
0,1(3) : 2022.11.23 20:26:29
0,1(4) : 2022.11.23 20:27:09
0,1(5) : 2022.11.23 20:27:50
0,1(6) : 2022.11.23 20:28:32
0,1(7) : 2022.11.23 20:29:13
0,1(8) : 2022.11.23 20:29:55
0,1(9) : 2022.11.23 20:30:37
0,1(10) : 2022.11.23 20:31:18
0,1(11) : 2022.11.23 20:32:00
0,1(12) : 2022.11.23 20:32:40


,1,2,3,4
Spain,25.3%,24.9%,24.4%,25.4%
CostaRica,25.3%,24.9%,24.4%,25.4%
Germany,43.1%,30.2%,19.7%,7.0%
Japan,8.8%,20.3%,30.9%,40.0%


In [17]:
#Group_E_RoundRobin.iat[0, 1], Group_E_RoundRobin.iat[1, 0] = '0,2' , '2,0'
Group_E_RoundRobin.iat[2, 3], Group_E_RoundRobin.iat[3, 2] = '3,0' , '0,3'

#Group_E_RoundRobin.iat[0, 2], Group_E_RoundRobin.iat[2, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 3], Group_E_RoundRobin.iat[3, 1] = '0,2' , '2,0'

#Group_E_RoundRobin.iat[0, 3], Group_E_RoundRobin.iat[3, 0] = '0,2' , '2,0'
#Group_E_RoundRobin.iat[1, 2], Group_E_RoundRobin.iat[2, 1] = '0,2' , '2,0'
Ranking(Group_E_RoundRobin)
Ranking_num_E, Ranking_probability_E = RankingProbability(Group_E_RoundRobin.copy())
for i in range(1,5):
    Ranking_JapanPropability.at['0,3', i] = Ranking_probability_E.at['Japan', i]
Ranking_probability_E

start : 2022.11.23 20:33:21
0,1(0) : 2022.11.23 20:33:21
0,1(1) : 2022.11.23 20:33:57
0,1(2) : 2022.11.23 20:34:33
0,1(3) : 2022.11.23 20:35:09
0,1(4) : 2022.11.23 20:35:50
0,1(5) : 2022.11.23 20:36:30
0,1(6) : 2022.11.23 20:37:11
0,1(7) : 2022.11.23 20:37:51
0,1(8) : 2022.11.23 20:38:32
0,1(9) : 2022.11.23 20:39:12
0,1(10) : 2022.11.23 20:39:52
0,1(11) : 2022.11.23 20:40:32
0,1(12) : 2022.11.23 20:41:13


,1,2,3,4
Spain,22.6%,27.5%,27.2%,22.7%
CostaRica,22.6%,27.5%,27.2%,22.7%
Germany,52.5%,29.1%,15.6%,2.9%
Japan,4.2%,16.3%,29.7%,49.9%


In [18]:
Ranking_JapanPropability

,1,2,3,4
"0,0",21.3%,27.7%,27.1%,24.0%
"1,1",21.7%,28.0%,26.6%,23.7%
"2,2",22.0%,28.3%,26.3%,23.4%
"3,0",52.5%,29.1%,15.6%,2.9%
"2,1",43.1%,30.2%,19.7%,7.0%
"2,0",47.4%,30.2%,17.7%,4.6%
"1,0",41.9%,30.4%,20.3%,7.4%
"0,1",8.4%,19.7%,30.7%,41.2%
"0,2",6.1%,17.8%,30.3%,45.8%
"1,2",8.8%,20.3%,30.9%,40.0%
